In [ ]:
# 0. INSTALLATION AND LIBRARY IMPORT
# ==============================================================================

!pip install astroquery -q
!pip install pandas numpy matplotlib seaborn scikit-learn tensorflow -q
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
import time
from astroquery.gaia import Gaia
from astroquery.vizier import Vizier
from tensorflow.keras.models import Model
from tensorflow.keras.layers import (
    Input, Dense, Dropout, BatchNormalization, Reshape,
    LayerNormalization, MultiHeadAttention, GlobalAveragePooling1D
)
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import classification_report, confusion_matrix, r2_score, mean_absolute_error
plt.style.use('seaborn-v0_8-whitegrid')
Gaia.MAIN_GAIA_TABLE = "gaiadr3.gaia_source"
Gaia.ROW_LIMIT = -1
Vizier.ROW_LIMIT = -1


In [ ]:
# ONCE YOU HAVE THE CSV DATA, RUN THIS PART
# ==============================================================================
# JUMP-START: LOAD DATASET WITHOUT RE-QUERY
# ==============================================================================
import pandas as pd
import numpy as np
import os

print("Starting session recovery (Recovery Mode)...")

# I often use Kaggle, so if you use Kaggle, you just need to change this part.

# Change the folder name
file_path = "/kaggle/input/final-master-dataset/FINAL_MASTER_DATASET.csv"

# 2. Check and Load
if os.path.exists(file_path):
    print(f"File found: {file_path}")
    print("Reading the CSV file, please wait a moment...")
    df_gabungan = pd.read_csv(file_path)

    print(f"Data reloaded successfully.")
    print(f"Number of rows: {len(df_gabungan):,}")
    print(f"Columns available: {list(df_gabungan.columns[:5])}...")

    # Quick check if Supergiant data exists
    if 'dataset_source' in df_gabungan.columns:
        print("\nData Source Check:")
        print(df_gabungan['dataset_source'].value_counts())

else:
    print(f"File not found at: {file_path}")
    print("Tips: If you have downloaded the file to your laptop, please use 'Add Data' -> 'Upload Dataset' at the top right.")

In [ ]:
# 1. QUERY FOR MAIN SEQUENCE AND SUBGIANT STARS

print("Fetching main data (Main Sequence, Subgiant, Giant)...")

query_utama_final = """
SELECT
    g.source_id, g.ra, g.dec,
    g.l, g.b,
    g.parallax, g.parallax_over_error,
    g.pmra, g.pmdec,
    g.phot_g_mean_mag, g.phot_bp_mean_mag, g.phot_rp_mean_mag,
    g.phot_bp_rp_excess_factor,
    g.ruwe, g.phot_variable_flag,

    -- FLAME & GSP-Phot PARAMETERS
    df.teff_gspphot, df.logg_gspphot, df.mh_gspphot,
    df.ag_gspphot, df.ebpminrp_gspphot,
    df.mg_gspphot,
    df.mass_flame, df.age_flame, df.evolstage_flame,
    df.flags_flame
FROM
    gaiadr3.gaia_source AS g
JOIN
    gaiadr3.astrophysical_parameters AS df
    ON g.source_id = df.source_id
WHERE
    g.parallax > 0.5 AND g.parallax_over_error > 10
    AND g.phot_g_mean_mag < 18
    AND g.phot_bp_mean_mag IS NOT NULL
    AND g.phot_rp_mean_mag IS NOT NULL
    AND g.phot_g_mean_mag < 18
    AND g.ruwe < 1.4
    AND df.flags_flame LIKE '0%'
    AND df.evolstage_flame IS NOT NULL
    AND df.mass_flame IS NOT NULL
    AND df.age_flame IS NOT NULL
    AND df.teff_gspphot BETWEEN 2500 AND 50000
    AND df.logg_gspphot BETWEEN -0.5 AND 6.0
    AND df.mh_gspphot > -5
    AND df.mg_gspphot BETWEEN -5 AND 15
    AND df.lum_flame IS NOT NULL
    AND g.random_index < 50000000
"""

try:
    job_utama = Gaia.launch_job_async(query_utama_final)
    df_utama = job_utama.get_results().to_pandas()
    print(f"Main data acquisition complete. Received {len(df_utama)} rows.")

    if 'l' in df_utama.columns:
        print(" -> Galactic columns (l, b) successfully retrieved.")
except Exception as e:
    print(f"Error occurred: {e}")

In [ ]:
# 2. QUERY FOR GIANT STARS

print("[Additional Query] Fetching SPECIFIC GIANT DATA...")

query_giant = """
SELECT
    g.source_id, g.ra, g.dec,
    g.l, g.b,
    g.parallax, g.parallax_over_error,
    g.pmra, g.pmdec,
    g.phot_g_mean_mag, g.phot_bp_mean_mag, g.phot_rp_mean_mag,
    g.phot_bp_rp_excess_factor,
    g.ruwe, g.phot_variable_flag,

    -- FLAME PARAMETERS
    df.teff_gspphot, df.logg_gspphot, df.mh_gspphot,
    df.ag_gspphot, df.ebpminrp_gspphot,
    df.mg_gspphot,
    df.mass_flame, df.age_flame, df.evolstage_flame,
    df.flags_flame
FROM
    gaiadr3.gaia_source AS g
JOIN
    gaiadr3.astrophysical_parameters AS df
    ON g.source_id = df.source_id
WHERE
    g.parallax > 0.1             -- Minimum distance (can be further than MS)
    AND g.parallax_over_error > 5
    AND g.phot_g_mean_mag < 18.5 -- Slightly dimmer magnitude allowed
    AND g.phot_bp_mean_mag IS NOT NULL
    AND g.phot_rp_mean_mag IS NOT NULL

    -- RELAXED FILTER SPECIFIC FOR GIANTS
    AND g.ruwe < 3.0             -- Giants have higher astrometric noise, so RUWE < 3.0 is acceptable
    -- REMOVED 'flags_flame' filter so 'difficult' Giants are included

    -- ONLY FETCH GIANTS (Base RGB to AGB)
    AND df.evolstage_flame BETWEEN 490 AND 1290

    -- Ensure Target Exists
    AND df.mass_flame IS NOT NULL
    AND df.age_flame IS NOT NULL

    -- Random Limit (Taking 50,000 Giants is sufficient for balancing)
    AND g.random_index < 20000000
"""

try:
    job_giant = Gaia.launch_job_async(query_giant)
    df_giant_raw = job_giant.get_results().to_pandas()

    # Initial Labeling
    df_giant_raw['dataset_source'] = 'Gaia_DR3_Giant_Query'
    df_giant_raw['evolutionary_phase'] = 'Giant'

    print(f"SUCCESS: Retrieved {len(df_giant_raw):,} Pure Giant Stars.")
    print(f"   Evolstage Range: {df_giant_raw['evolstage_flame'].min()} - {df_giant_raw['evolstage_flame'].max()}")

except Exception as e:
    print(f"Failed to Query Giants: {e}")
    df_giant_raw = pd.DataFrame()

In [ ]:
# 3. WHITE DWARF DATA ACQUISITION (STANDARD ASTROQUERY METHOD)
# ==============================================================================
print("Fetching White Dwarf Catalog (Gentile Fusillo et al. 2021)...")

from astroquery.vizier import Vizier
import pandas as pd
import numpy as np

# 1. Vizier Configuration: Request all columns ["**"]
# Ensures critical columns like AgeH/MassH are not hidden
v = Vizier(row_limit=50000, columns=["**"])

df_wd_raw = pd.DataFrame()

try:
    # 2. Retrieve tables from Catalog ID
    catalogs = v.get_catalogs("J/MNRAS/508/3877")
    print(f"   -> Found {len(catalogs)} tables.")

    # 3. Find the main table (the one with the most columns)
    target_catalog = None
    max_cols = 0

    for i, table in enumerate(catalogs):
        # Convert to Pandas first for safe column checking
        temp_df = table.to_pandas()
        print(f"      - Table {i}: {len(temp_df)} rows, {len(temp_df.columns)} columns")

        if len(temp_df.columns) > max_cols:
            max_cols = len(temp_df.columns)
            target_catalog = temp_df

    if target_catalog is not None:
        df_wd_raw = target_catalog
        print(f"Selected table! Total: {len(df_wd_raw)} rows.")

        # Debug: Check for existence of Age/Mass columns
        check_cols = [c for c in df_wd_raw.columns if 'Age' in c or 'Mass' in c]
        print(f"   -> Target columns detected: {check_cols[:5]}...")
    else:
        raise ValueError("No valid table found.")

except Exception as e:
    print(f"Failed to retrieve VizieR data: {e}")

# ==============================================================================
# 4. WD DATA HARMONIZATION (ROBUST CLEANING)
# ==============================================================================
def process_gentile_data_robust(df):
    if df.empty: return df

    data = df.copy()

    # A. White Dwarf Probability Filter (PWD > 0.95)
    pwd_cols = [c for c in data.columns if 'PWD' in c.upper()]
    if pwd_cols:
        col_pwd = pwd_cols[0]
        print(f"   -> Filtering PWD using column '{col_pwd}' > 0.95")
        data = data[data[col_pwd] > 0.95].copy()

    # B. Column Name Mapping
    mapping_candidates = {
        'source_id': ['Source', 'EDR3', 'GaiaEDR3'],
        'ra': ['RA_ICRS', 'RA'],
        'dec': ['DE_ICRS', 'DE'],
        'parallax': ['Plx'],
        'parallax_error': ['e_Plx'],
        'phot_g_mean_mag': ['Gmag'],
        'phot_bp_mean_mag': ['BPmag'],
        'phot_rp_mean_mag': ['RPmag'],
        'mass_wd': ['MassH', 'Mass', 'mass'],       # Target 1 (Mass)
        'age_wd_cooling': ['AgeH', 'Age', 'age'],   # Target 2 (Age)
        'teff_wd': ['TeffH', 'Teff'],
        'logg_wd': ['loggH', 'logg'],
        'ruwe': ['RUWE']
    }

    rename_dict = {}
    for target_col, candidates in mapping_candidates.items():
        for cand in candidates:
            if cand in data.columns:
                rename_dict[cand] = target_col
                break
    data.rename(columns=rename_dict, inplace=True)

    # B2. Safety Net for Age
    if 'age_wd_cooling' not in data.columns:
        fuzzy_age = [c for c in data.columns if "age" in c.lower()]
        if fuzzy_age:
            print(f"   i Auto-recover age column: Using '{fuzzy_age[0]}'")
            data.rename(columns={fuzzy_age[0]: 'age_wd_cooling'}, inplace=True)

    # C. Feature Engineering
    if 'parallax' in data.columns and 'parallax_error' in data.columns:
        data['parallax_over_error'] = data['parallax'] / data['parallax_error']
    else:
        data['parallax_over_error'] = np.nan

    if 'phot_bp_mean_mag' in data.columns and 'phot_rp_mean_mag' in data.columns:
        data['bp_rp0'] = data['phot_bp_mean_mag'] - data['phot_rp_mean_mag']
    else:
        data['bp_rp0'] = np.nan

    if 'phot_g_mean_mag' in data.columns and 'parallax' in data.columns:
        with np.errstate(divide='ignore'):
            dist_pc = 1000.0 / data['parallax']
            data['abs_G0'] = data['phot_g_mean_mag'] - 5 * np.log10(dist_pc) + 5
    else:
        data['abs_G0'] = np.nan

    # D. Labeling
    data['evolutionary_phase'] = 'White Dwarf'
    data['dataset_source'] = 'GentileFusillo2021'

    # E. Final Validation
    if 'mass_wd' in data.columns and 'age_wd_cooling' in data.columns:
        before = len(data)
        data = data.dropna(subset=['mass_wd', 'age_wd_cooling'])
        after = len(data)
        print(f"   -> COMPLETE Data (Mass & Age): {after} (Dropped {before-after})")
    else:
        print("CRITICAL: Target Mass/Age columns missing!")
        print("   Available columns:", data.columns.tolist())

    return data

# --- EXECUTE ---
if not df_wd_raw.empty:
    df_wd_clean = process_gentile_data_robust(df_wd_raw)
    print(f"Final WD Data Ready: {len(df_wd_clean)} rows.")
else:
    print("Warning: Raw WD data is empty.")

In [ ]:
# 5. SCIENTIFIC PAPER CROSS-MATCH (DEBUGGED & FIXED)
# ==============================================================================
import pandas as pd
import numpy as np
import time
import re
from astroquery.vizier import Vizier
from astroquery.gaia import Gaia
from astropy.coordinates import SkyCoord
from astropy import units as u

print("STARTING SCIENTIFIC LITERATURE-BASED CROSS-MATCH (REVISED)...")

# --- COORDINATE CONVERSION HELPER FUNCTION ---
def clean_coord(val):
    """Forces conversion to float. Returns NaN if value is sexagesimal string (dropped later)."""
    try:
        return float(val)
    except:
        return np.nan

# ------------------------------------------------------------------------------
# 1. DEFINE TARGET CATALOGS
# ------------------------------------------------------------------------------
# We use IDs that are guaranteed to have data (Stable)
target_papers = [
    {
        "name": "Pantaleoni (ALS II)",
        "id": "J/MNRAS/504/2968",  # 2021 Version (Stable & Complete)
        "type": "Blue Supergiant/OB"
    },
    {
        "name": "Messineo (RSG)",
        "id": "J/AJ/158/20",       # 2019 Version (Abundant & Stable)
        "type": "Red Supergiant"
    },
    {
        "name": "Zhang (RSG)",
        "id": "J/ApJ/889/33",      # Zhang et al. 2020 (RSG Catalog)
        "type": "Red Supergiant"
    }
]

# ------------------------------------------------------------------------------
# 2. VIZIER DATA RETRIEVAL LOOP
# ------------------------------------------------------------------------------
print("\n[1/3] Fetching Coordinates from Vizier (Unlimited Rows)...")

# IMPORTANT: Set row_limit=-1 HERE to retrieve all available data
v = Vizier(columns=["**"], row_limit=-1)

reference_stars = []

for paper in target_papers:
    print(f"   -> Attempting catalog: {paper['name']} ({paper['id']})...", end=" ")

    try:
        cats = v.get_catalogs(paper['id'])

        # Find the largest table (usually the main dataset)
        df_res = pd.DataFrame()
        max_rows = 0
        for c in cats:
            if len(c) > max_rows:
                df_res = c.to_pandas()
                max_rows = len(c)

        if df_res.empty:
            print("Empty.")
            continue

        print(f"Success ({len(df_res)} rows).")

        # --- AUTO-DETECT RA/DEC ---
        cols = df_res.columns.tolist()

        # Search for RA/DEC columns (prioritize decimal formats: RAJ2000, RA_ICRS, RAdeg)
        c_ra = next((c for c in cols if c in ['RAJ2000', 'RA_ICRS', '_RA', 'RAdeg', 'RA']), None)
        c_dec = next((c for c in cols if c in ['DEJ2000', 'DE_ICRS', '_DE', 'DEdeg', 'DE']), None)

        # Search for Mass column (Specific to Pantaleoni)
        c_mass = next((c for c in cols if c in ['Mass', 'Mini', 'M_evol', 'M']), None)

        if c_ra and c_dec:
            # Standardization
            temp = pd.DataFrame()

            # IMPORTANT: Force conversion to Float (Decimal)
            # Vizier sometimes returns strings like "18 20 30" -> This causes SQL Errors
            # We attempt simple conversion. If it fails, we use Astropy SkyCoord (slower but robust)

            # Check sample data type
            sample_val = df_res[c_ra].iloc[0]
            if isinstance(sample_val, str) and " " in sample_val:
                # This is hour/minute format (sexagesimal). Convert using SkyCoord
                print("      Warning: String coordinate format detected. Converting...", end=" ")
                coords = SkyCoord(df_res[c_ra].astype(str), df_res[c_dec].astype(str),
                                  unit=(u.hourangle, u.deg), frame='icrs')
                temp['ra_ref'] = coords.ra.deg
                temp['dec_ref'] = coords.dec.deg
                print("Done.")
            else:
                # Assume decimal
                temp['ra_ref'] = pd.to_numeric(df_res[c_ra], errors='coerce')
                temp['dec_ref'] = pd.to_numeric(df_res[c_dec], errors='coerce')

            temp['paper_source'] = paper['name']
            temp['star_type'] = paper['type']

            if c_mass:
                temp['mass_ref'] = pd.to_numeric(df_res[c_mass], errors='coerce')
            else:
                temp['mass_ref'] = np.nan

            # Drop rows where coordinate conversion failed
            temp = temp.dropna(subset=['ra_ref', 'dec_ref'])
            reference_stars.append(temp)
        else:
            print(f"      Skip: Coordinate columns not recognized in {cols[:5]}...")

    except Exception as e:
        print(f"Error: {e}")

# Concatenate
if reference_stars:
    df_refs = pd.concat(reference_stars, ignore_index=True)
    print(f"\nTotal Valid Targets: {len(df_refs)} Stars.")
else:
    print("Failed to aggregate reference stars.")
    df_refs = pd.DataFrame()

# ------------------------------------------------------------------------------
# 3. GAIA DR3 CROSS-MATCH (WITH ERROR CHECKING)
# ------------------------------------------------------------------------------
if not df_refs.empty:
    print("\n[2/3] Cross-Matching with Gaia DR3 (Batch Query)...")

    gaia_results = []
    targets = df_refs.to_dict('records')
    batch_size = 50

    for i in range(0, len(targets), batch_size):
        batch = targets[i:i+batch_size]

        # Query Construction
        conditions = []
        for t in batch:
            # Ensure coordinates are valid floats
            if pd.notna(t['ra_ref']) and pd.notna(t['dec_ref']):
                cond = f"1=CONTAINS(POINT('ICRS', ra, dec), CIRCLE('ICRS', {t['ra_ref']:.5f}, {t['dec_ref']:.5f}, 0.00028))"
                conditions.append(cond)

        if not conditions: continue

        where_clause = " OR ".join(conditions)

        query = f"""
        SELECT
            source_id, ra, dec, parallax, parallax_over_error,
            phot_g_mean_mag, phot_bp_mean_mag, phot_rp_mean_mag, bp_rp,
            ruwe, phot_g_mean_flux_over_error,
            teff_gspphot, logg_gspphot, mh_gspphot
        FROM gaiadr3.gaia_source
        WHERE ({where_clause})
          AND phot_g_mean_mag < 19
        """

        try:
            job = Gaia.launch_job(query)
            res = job.get_results().to_pandas()

            if not res.empty:
                # Reverse Match
                for _, g_row in res.iterrows():
                    best_match = None
                    min_dist = 1.0

                    for t_row in batch:
                        d = np.sqrt((g_row['ra'] - t_row['ra_ref'])**2 + (g_row['dec'] - t_row['dec_ref'])**2)
                        if d < min_dist:
                            min_dist = d
                            best_match = t_row

                    if min_dist < 0.0004 and best_match:
                        row_data = g_row.to_dict()
                        row_data['paper_source'] = best_match['paper_source']
                        row_data['star_type'] = best_match['star_type']
                        row_data['mass_ref'] = best_match['mass_ref']

                        # Evolutionary Phase Labeling (Int) for Neural Network
                        # 0: Blue/OB, 1: Red SG
                        row_data['label_class'] = 0 if 'Blue' in best_match['star_type'] else 1

                        gaia_results.append(row_data)

            print(f"\r   -> Batch {i//batch_size + 1}: Found {len(res)} candidates...", end="")

        except Exception as e:
            # DISPLAY ERROR IF FAILED (Debugging purpose)
            print(f"\n   Batch Error: {e}")
            time.sleep(1)

    print(f"\n\nDone! Found {len(gaia_results)} valid matches.")

    # 4. SAVE
    if gaia_results:
        df_final = pd.DataFrame(gaia_results).drop_duplicates(subset='source_id')

        # Calculate Abs Mag
        df_final['G_abs'] = df_final['phot_g_mean_mag'] + 5 * np.log10(df_final['parallax']/1000.0) + 5

        print("\nFinal Statistics:")
        print(df_final['paper_source'].value_counts())

        filename = "gaia_scientific_supergiants_fixed.csv"
        df_final.to_csv(filename, index=False)
        print(f"File saved: {filename}")
else:
    print("No data available.")

In [ ]:
# 6. FINAL DATA MERGING (REVISED: 4 SOURCES INCLUDING GIANT QUERY)
# ==============================================================================
import pandas as pd
import numpy as np

print("Merging 4 Datasets (Main + Giant + Scientific SG + WD)...")

frames = []

# --- A. MAIN DATASET (Main Sequence/Subgiant) ---
if 'df_utama' in locals() and not df_utama.empty:
    df_u = df_utama.copy()
    df_u['dataset_source'] = 'Gaia_DR3_Main'
    # We focus this dataset on MS & Subgiant, leaving Giants to be handled by the specific query
    # (Optional: Filter < 490 if strict cleanliness is needed, but deduplication handles it)

    # Feature Calculation
    df_u['bp_rp0'] = df_u['phot_bp_mean_mag'] - df_u['phot_rp_mean_mag']
    with np.errstate(divide='ignore', invalid='ignore'):
        df_u['abs_G0'] = df_u['phot_g_mean_mag'] - 5*np.log10(1000.0/df_u['parallax']) + 5

    df_u['mass_wd'] = np.nan
    df_u['age_wd_cooling'] = np.nan

    frames.append(df_u)
    print(f"   -> [Main] {len(df_u):,} stars ready.")

# --- B. SPECIFIC GIANT DATASET (Additional Query) ---
if 'df_giant_raw' in locals() and not df_giant_raw.empty:
    df_g = df_giant_raw.copy()
    df_g['dataset_source'] = 'Gaia_DR3_Giant_Query'

    # Feature Calculation
    df_g['bp_rp0'] = df_g['phot_bp_mean_mag'] - df_g['phot_rp_mean_mag']
    with np.errstate(divide='ignore', invalid='ignore'):
        df_g['abs_G0'] = df_g['phot_g_mean_mag'] - 5*np.log10(1000.0/df_g['parallax']) + 5

    df_g['mass_wd'] = np.nan
    df_g['age_wd_cooling'] = np.nan

    frames.append(df_g)
    print(f"   -> [Giant Query] {len(df_g):,} stars ready.")

# --- C. SUPERGIANT DATASET (SCIENTIFIC - High Priority) ---
try:
    if 'df_final' in locals() and 'paper_source' in df_final.columns:
        df_sg_sci = df_final.copy()
    else:
        df_sg_sci = pd.read_csv("gaia_scientific_supergiants_fixed.csv")

    df_sg_sci['dataset_source'] = 'Scientific_Paper_SG'

    # Dummy Code for SG
    df_sg_sci['evolstage_flame'] = 400

    if 'mass_ref' in df_sg_sci.columns:
        df_sg_sci.rename(columns={'mass_ref': 'mass_flame'}, inplace=True)

    if 'bp_rp0' not in df_sg_sci.columns:
        if 'bp_rp' in df_sg_sci.columns:
             df_sg_sci['bp_rp0'] = df_sg_sci['bp_rp']
        else:
             df_sg_sci['bp_rp0'] = df_sg_sci['phot_bp_mean_mag'] - df_sg_sci['phot_rp_mean_mag']

    if 'abs_G0' not in df_sg_sci.columns and 'G_abs' in df_sg_sci.columns:
         df_sg_sci['abs_G0'] = df_sg_sci['G_abs']

    if 'age_flame' not in df_sg_sci.columns:
        df_sg_sci['age_flame'] = np.nan

    frames.append(df_sg_sci)
    print(f"   -> [Scientific SG] {len(df_sg_sci):,} stars (Gold Standard).")

except Exception as e:
    print(f"Warning: Scientific SG dataset not found ({e}).")

# --- D. WHITE DWARF DATASET (Gentile Fusillo) ---
if 'df_wd_clean' in locals() and not df_wd_clean.empty:
    df_w = df_wd_clean.copy()

    if 'RUWE' in df_w.columns:
        df_w.rename(columns={'RUWE': 'ruwe'}, inplace=True)

    df_w['mass_flame'] = np.nan
    df_w['age_flame'] = np.nan
    df_w['evolstage_flame'] = 500

    frames.append(df_w)
    print(f"   -> [White Dwarf] {len(df_w):,} stars ready.")


# --- E. EXECUTE MERGE & INTELLIGENT DEDUPLICATION ---
if frames:
    df_gabungan = pd.concat(frames, ignore_index=True)

    # Priority Logic (Lower number = Higher Priority)
    # 1. Scientific Paper (Human Verified) -> Most Valid
    # 2. White Dwarf (Specific Catalog)
    # 3. Giant Query (Specific Catalog)
    # 4. Main Dataset (General)
    source_priority = {
        'Scientific_Paper_SG': 0,
        'Vizier_Pantaleoni2021': 0,
        'Vizier_Hohle2010': 0,
        'GentileFusillo2021': 1,
        'Gaia_DR3_Giant_Query': 2,
        'Gaia_DR3_Main': 3
    }

    df_gabungan['prio'] = df_gabungan['dataset_source'].map(source_priority).fillna(99)
    df_gabungan.sort_values('prio', inplace=True)

    # Remove Duplicates based on Source ID
    before_dedup = len(df_gabungan)
    df_gabungan.drop_duplicates(subset=['source_id'], keep='first', inplace=True)
    after_dedup = len(df_gabungan)

    print(f"\nDeduplication Complete: {before_dedup - after_dedup} duplicates removed.")

    # Final Quality Filter
    df_gabungan = df_gabungan[
        (df_gabungan['parallax'] > 0) &
        (df_gabungan['bp_rp0'].notnull()) &
        (df_gabungan['abs_G0'].notnull())
    ].copy()

    # Shuffle Data
    df_gabungan = df_gabungan.sample(frac=1, random_state=42).reset_index(drop=True)

    print("\nFINAL MASTER DATASET READY!")
    print(f"Total Rows: {len(df_gabungan):,}")
    print("-" * 30)
    print("Data Source Distribution:")
    print(df_gabungan['dataset_source'].value_counts())
    print("-" * 30)

    # Check for True Massive Stars (> 10 M_sun)
    monsters = len(df_gabungan[df_gabungan['mass_flame'] > 10.0])
    print(f"True Massive Stars (> 10 M_sun): {monsters}")

    # Save to Master CSV
    df_gabungan.to_csv("FINAL_MASTER_DATASET.csv", index=False)

else:
    print("Failed to create df_gabungan.")